# 🚢 Snowflake to Model Deployment Demo

In this demo, you'll walk through a complete machine learning pipeline—from data ingestion to deployment and inference—using containerized infrastructure.

## 📝 Prerequisites

Before starting, please ensure the following:

- Load the `titanic_snowflake.csv` dataset into your notebook environment.

Once the data is loaded, the notebook is designed to run **top-down** without interruption.

---

## 🔹 Demo Overview

This demo includes the following key steps:

1. **Data Ingestion from Snowflake**  
   Pull structured Titanic dataset from Snowflake.

2. **Feature Engineering**  
   Transform raw data into meaningful features for model training.

3. **Model Training with XGBoost**  
   Use XGBoost to train a classification model on the engineered dataset.

4. **Model Deployment**  
   Register and deploy the trained model.

5. **Batch Inference**  
   Call the deployed model to make predictions on new batches of data.


In [ ]:
# Not neccessary since these packages come with the runtime (Just an example)
#!pip install xgboost snowflake-ml-python 

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from snowflake.ml.registry import Registry
import ast
#add another package
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
titanic = pd.read_csv('data/titanic_snowflake.csv')
titanic = titanic.drop(["AGE", 
                        "DECK", 
                        "ALIVE",
                        "ADULT_MALE",
                        "EMBARKED",
                        "PCLASS",
                        "ALONE",
                        "SEX"],axis=1)
titanic.head()

Usually your data will already be in Snowflake.  This next step shows how to write the pandas dataframe as a table, then how to turn a table from Snowflake into a pandas dataframe

In [ ]:
# This step turns pandas -> snowpark and writes to snowflake
titanic_sf = session.create_dataframe(titanic)
titanic_sf.write.mode("overwrite").save_as_table("titanic_raw")

In [ ]:
# Here we read a table from Snowflake into a Snowpark dataframe

titanic_raw = session.table('titanic_raw').to_pandas()
titanic_raw.head()

In [ ]:
titanic.dropna(inplace=True)

In [ ]:
titanic = pd.get_dummies(titanic, drop_first=True)

# Convert all boolean columns to integers
titanic = titanic.apply(lambda x: x.astype(int) if x.dtype == 'bool' else x)

titanic.dtypes

In [ ]:
x = titanic.drop('SURVIVED',axis=1)
y = titanic.SURVIVED

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=.70,random_state=1234)

In [ ]:
param_grid = {
    "n_estimators": [100, 200],
    "learning_rate": [0.1, 0.5],
    "max_depth": [1,2,3,4,5,6],
    "min_child_weight": [1, 6]
}

In [ ]:
model = XGBClassifier(objective='binary:logistic', 
                      eval_metric='logloss')

grid_search = GridSearchCV(estimator=model, 
                           param_grid=param_grid)

grid_search.fit(xtrain, ytrain)

In [ ]:
# Best parameters and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
test_score = best_model.score(xtest, ytest)
print("Test Score:", test_score)

In [ ]:
metrics = {
    "Accuracy": best_score,
    "Params": best_params
}

metrics

In [ ]:
from snowflake.ml.registry import Registry

# Get sample input data to pass into the registry logging function
X = xtrain.sample(n=1)

# Create a registry and log the model
# You can specify a different DB and Schema if you'd like
# otherwise it uses the session context
# If a registry does not exist it will create one
reg = Registry(session=session)

# Define model name and version (use uppercase for name)
model_name = "TITANIC_SERVICE"

titanic_model = reg.log_model(
    model_name=model_name,
    options = {
    "relax_version": True,
    },
    target_platforms=["SNOWPARK_CONTAINER_SERVICES"],
    #version_name="V_1", # If you leave version_name off SF creates one
    model=best_model,
    sample_input_data=X,
    metrics=metrics,
)

In [ ]:
models_df = reg.show_models()
models_df[models_df['name'] == model_name]

In [ ]:
models = reg.get_model(model_name).show_versions()
models.sort_values(by='created_on', ascending=False)

In [ ]:
recent_model = reg.get_model(model_name).last()
recent_model

In [ ]:
m = reg.get_model(model_name).last()
m.default = m
mv = m.default
mv.version_name

In [ ]:
-- If you do not have an image repo create on
CREATE IMAGE REPOSITORY IF NOT EXISTS tutorial_repository;

### Deploying a Model to Snowpark Container Services as a Long-Running Service

This section explains how to deploy a machine learning model to Snowpark Container Services (SPCS) using Model Serving. The deployed service will run continuously and expose a REST API endpoint for prediction.

If you're currently using `system_compute_pool_CPU`, you will need to create a separate compute pool to host the service. 

> ⚠️ You may need `SYSADMIN` privileges to create a compute pool.

```sql
CREATE COMPUTE POOL tutorial_compute_pool
  MIN_NODES = 1
  MAX_NODES = 1
  INSTANCE_FAMILY = CPU_X64_XS;
'''

In [ ]:
-- If you do not have a compute pool create one
CREATE COMPUTE POOL IF NOT EXISTS titanic_compute_pool
  MIN_NODES = 1
  MAX_NODES = 2
  INSTANCE_FAMILY = CPU_X64_M;

In [ ]:
image_repo_name = "tutorial_repository"

cp_name = "titanic_compute_pool"
num_spcs_nodes = '1'
service_name = 'TITANIC_PREDICTION_SERVICE'

current_database = session.get_current_database().replace('"', '')
current_schema = session.get_current_schema().replace('"', '')
extended_image_repo_name = f"{current_database}.{current_schema}.{image_repo_name}"
extended_service_name = f'{current_database}.{current_schema}.{service_name}'

In [ ]:
DROP SERVICE IF EXISTS {{service_name}};

In [ ]:
mv.create_service(
    service_name=extended_service_name,
    service_compute_pool=cp_name,
    image_repo=extended_image_repo_name,
    ingress_enabled=True,
    max_instances=int(num_spcs_nodes),
    build_external_access_integration="ALLOW_ALL_INTEGRATION"
)

In [ ]:
-- Show the compute pool has a service
describe compute pool titanic_compute_pool;

In [ ]:
SHOW SERVICES LIKE '%TITANIC_PREDICTION_SERVICE%';

In [ ]:
# Can also view this in the Model Registry UI
mv.list_services()

In [ ]:
test_sf = session.create_dataframe(xtest)

In [ ]:
mv.run(test_sf, 
            function_name = "PREDICT", 
            service_name = "CROMANO.DEMO.TITANIC_PREDICTION_SERVICE")

Since we created a REST API above, this service will run continuously. It is a good idea to drop or suspend the service if you do not need it. Compute pool will automatically suspend if no service is running.

## Make sure to stop the service at the end of the demo so it does not stay on

In [ ]:
-- ALTER SERVICE {{service_name}} SUSPEND;